**Toronto Clustering**

First we scrape Toronto neighborhood data from Wikipedia. This is a horrible thing to do Wikipedia is a horrible source of data, and the pages could change any time.  But it is what the instructions ask for, so we will try to bite our tongue and carry on.  

In [1]:
import pandas as pd
import numpy as np
import requests



In [2]:
# Retrieve the raw html wiki page
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
raw_page = requests.get(wiki_url)

In [3]:
# We are assuming here that the table we want is the first on the page.
# This is horrible, but about the best we can do scraping a Wikipedia page
df = pd.read_html(raw_page.text)[0]

In [4]:
# Now we remove rows without Boroughs

df_w_boroughs = df[df["Borough"] != "Not assigned"]


In [5]:
# Replace neighbourhoods that are not assigned with their borough
# This fixes one entry, Queen's Park
for index, row in df_w_boroughs.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]
        
        

In [6]:
# fix the index
df_w_boroughs.reset_index(inplace=True, drop=True)

In [7]:

# This assumes no two boroughs share postal codes, which is true
# This also clusters the Borough, which wasn't asked for, but seems harmless
gp = df_w_boroughs.groupby(["Postcode", "Borough"], as_index=False)
# Join the neighborhoods in the same Postcode in a comma seperated list
df_final = gp.agg({"Neighbourhood": ", ".join})

In [8]:
df_final.describe()

,Postcode,Borough,Neighbourhood
count,103,103,103
unique,103,11,102
top,M5T,North York,Queen's Park
freq,1,24,2


In [9]:
df_final.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
df_final.shape

(103, 3)